In [1]:
import pandas as pd
import numpy as np
import requests
import json
import time
from scipy.optimize import linear_sum_assignment

In [2]:
input_csv=input('Enter the csv file name: ')

Enter the csv file name:  central_bus_data


In [3]:
bus_df=pd.read_csv(f'{input_csv}.csv')
bus_df.head()

,Sl No,College/Institute,Vendor Name,Vehicle Number,Route No,Driver Point Latitude,Driver Point Longitude,Driver Point Name,First Pickup Pt. Latitude,First Pickup Pt. Longitude,First Pickup Point Name,Seat Capacity
0,216,Amity University,Baghirathi,KA51AJ8295,7,12.987582,77.600954,Shivaji Nagar,13.028943,77.567725,Ramaiah Hospital,33
1,218,AmityUniversity,Baghirathi,KA51AJ8298,4,13.027588,77.710653,KR Puram,13.008776,77.692368,KR Puram,33
2,219,Amity University,Baghirathi,KA51AJ8291,6,13.032832,77.609844,Shivaji Nagar,12.964512,77.649514,Leela Palace Rd,33
3,220,Amity University,Baghirathi,KA51AJ8296,8,13.091024,77.580596,Goraguntepalya,13.039833,77.519832,Jalahalli,33
4,221,Amity University,Baghirathi,KA53AC0344,5,13.012663,77.607487,Kadubeesanahall,12.934740,77.690393,"Kadubeesanahalli, Opp. Wells Fargo",53


In [4]:
cluster_name=input('Name of the current Cluster: ')

Name of the current Cluster:  CENTRAL_phase1


In [6]:
driver_df=bus_df[['Vehicle Number', 'Route No', 'Driver Point Latitude', 'Driver Point Longitude', 'Driver Point Name', 'College/Institute']].rename(columns={
    'Vehicle Number':'bus', 
    'Route Number': 'route',
    'Driver Point Latitude': 'dlat',
    'Driver Point Longitude': 'dlon',
    'Driver Point Name': 'dname',
    'College/Institute': 'cname'
})
pickup_df=bus_df[['Vehicle Number', 'Route No', 'First Pickup Pt. Latitude', 'First Pickup Pt. Longitude', 'First Pickup Point Name', 'College/Institute']].rename(columns={
    'Vehicle Number':'bus', 
    'Route Number': 'route',
    'First Pickup Pt. Latitude': 'plat',
    'First Pickup Pt. Longitude': 'plon',
    'First Pickup Point Name': 'pname',
    'College/Institute': 'cname'
})

In [7]:
driver_df.head()
driver_df.set_index('bus')

,Route No,dlat,dlon,dname,cname
bus,,,,,
KA51AJ8295,7,12.987582,77.600954,Shivaji Nagar,Amity University
KA51AJ8298,4,13.027588,77.710653,KR Puram,AmityUniversity
KA51AJ8291,6,13.032832,77.609844,Shivaji Nagar,Amity University
KA51AJ8296,8,13.091024,77.580596,Goraguntepalya,Amity University
KA53AC0344,5,13.012663,77.607487,Kadubeesanahall,Amity University
...,...,...,...,...,...
KA53AB1810,1,12.996135,77.757787,whitefield,Harrow
KA50A5968,2,13.017085,77.657374,banaswadi,Harrow
KA53AB1811,3,12.985176,77.601569,Shivaji Nagar,Harrow


In [8]:
pickup_df.head()
pickup_df.set_index('bus')

,Route No,plat,plon,pname,cname
bus,,,,,
KA51AJ8295,7,13.028943,77.567725,Ramaiah Hospital,Amity University
KA51AJ8298,4,13.008776,77.692368,KR Puram,AmityUniversity
KA51AJ8291,6,12.964512,77.649514,Leela Palace Rd,Amity University
KA51AJ8296,8,13.039833,77.519832,Jalahalli,Amity University
KA53AC0344,5,12.934740,77.690393,"Kadubeesanahalli, Opp. Wells Fargo",Amity University
...,...,...,...,...,...
KA53AB1810,1,13.030863,77.665129,Oakwood aptr,Harrow
KA50A5968,2,13.020205,77.656504,Horamavu Signal,Harrow
KA53AB1811,3,12.950439,77.642715,Domlur EGL,Harrow


In [9]:
base_url= "http://localhost:5000/route/v1/driving"

In [10]:
def osrm_calculate(lat1, lon1, lat2, lon2): #input parameters as default lat,lon
    src=f'{lon1},{lat1};{lon2},{lat2}'
    curl=f"http://localhost:5000/route/v1/driving/{src}?overview=false"
    time.sleep(0.2)
    try:
        json_response=requests.get(curl).json()
        return json_response['routes'][0]['distance']/1000
    except Exception as exc:
        print(f"OSRM error at {lat1},{lon1} to {lat2},{lon2}: {exc}")
        return np.inf

In [11]:
buses=bus_df['Vehicle Number'].values
buses

array(['KA51AJ8295', 'KA51AJ8298', 'KA51AJ8291', 'KA51AJ8296',
       'KA53AC0344', 'KA53AC0345', 'KA51B9664', 'KA53AB4252', 'KA439840',
       'KA43A0245', 'KA439841', 'KA439842', 'KA53B6963', 'KA53B1064',
       'KA53D0153', 'KA53B1072', 'KA53B0960', 'KA53B0958', 'KA53B0952',
       'KA53B1061', 'KA53B1075', 'KA53AC0540', 'KA53AC0634', 'KA07B1355',
       'KA53D8480', 'KA53D8723', 'KA53D8726', 'KA53AB9107', 'KA53D8404',
       'KA53D8473', 'KA53D8547', 'KA53D8727', 'KA07B1354', 'KA53D8471',
       'KA53D8405', 'KA53D8403', 'KA53D8722', 'KA53AB9114', 'KA51AJ8288',
       'KA53D8543', 'KA53D8472', 'KA53D8728', 'KA53D5167', 'KA53D8720',
       'KA53D8540', 'KA53D8537', 'KA53D8402', 'KA53D8399', 'KA53D8549',
       'KA53D2722', 'KA53D8550', 'KA53D4604', 'KA53D2898', 'KA53D8551',
       'KA53D8729', 'KA53D8541', 'KA53D5164', 'KA53D8401', 'KA53D2720',
       'KA53D8539', 'KA53D8542', 'KA53D4605', 'KA53D2957', 'KA53D2899',
       'KA53AB9116', 'KA53D8400', 'KA53D8477', 'KA53D2897', 'KA53D27

In [14]:
bus_coords = driver_df.set_index('bus')[['dlat', 'dlon']].to_dict('index')
buses = list(bus_coords.keys())

pickup_coords = pickup_df.set_index('bus')[['plat', 'plon']].to_dict('index')
pickups = list(pickup_coords.keys())

distance_mat=pd.DataFrame(index=buses, columns=pickups, dtype=float)

In [ ]:
for bus in buses:
    for pickup in buses:
        distance=osrm_calculate(bus_coords[bus]['dlat'], bus_coords[bus]['dlon'], pickup_coords[pickup]['plat'], pickup_coords[pickup]['plon'])
        distance_mat.loc[bus, pickup] =distance 
distance_mat

In [ ]:
distance_mat.to_csv('NPS_dist_matrix.csv', index=False)

In [ ]:
dist_mat_name=input('Distance matrix name: ')

In [ ]:
distance_mat=pd.read_csv(f'{dist_mat_name}.csv')

In [ ]:
distance_mat

In [ ]:
optim_drivers, optim_pickups=linear_sum_assignment(distance_mat)
driver = distance_mat.index[optim_drivers]
pickup = distance_mat.index[optim_pickups]

In [ ]:
 driver

In [ ]:
optimized_df = pd.DataFrame({
    'From Bus': distance_mat.columns[optim_drivers],
    'Driver Site':driver_df.loc[driver,'cname'].values,
    'Driver pt lat': driver_df.loc[driver,'dlat'].values,
    'Driver pt long': driver_df.loc[driver,'dlon'].values,
    'Driver pt name': driver_df.loc[driver,'dname'].values,
    'Driver Route': driver_df.loc[driver,'route'].values,
    'To Bus': distance_mat.columns[optim_pickups],
    'Pickup Route': pickup_df.loc[pickup,'route'].values,
    'Pickup pt name': pickup_df.loc[pickup,'pname'].values,
    'Pickup pt lat': pickup_df.loc[pickup,'plat'].values,
    'Pickup pt long': pickup_df.loc[pickup,'plon'].values,
    'Pickup Site':pickup_df.loc[pickup,'cname'].values,
    'Optimized dead km': distance_mat.values[optim_drivers, optim_pickups],
    'Original dead km': distance_mat.values[optim_drivers, optim_drivers]
})

In [ ]:
optimized_df

In [ ]:
optimized_df.to_excel(f'{cluster_name}_Optimal.xlsx', index=False)

In [ ]:
def find_swap_chains(from_buses, to_buses):
    assignments = dict(zip(from_buses, to_buses))
    seen = set()
    chains = []

    for start in from_buses:
        if start not in seen:
            chain = []
            current = start
            while current not in seen and current in assignments:
                chain.append(current)
                seen.add(current)
                current = assignments[current]
                if current == chain[0]:  # Closed loop
                    chain.append(current)
                    break
            if len(chain) > 1:
                chains.append(chain)
    return chains


In [ ]:
chains = find_swap_chains(optimized_df['From Bus'], optimized_df['To Bus'])

for i, chain in enumerate(chains, 1):
    print(f"Swap Chain {i}:", " → ".join(chain))


In [ ]:
chain_data = []
cleaned_chains = [[bus.strip() for bus in chain] for chain in chains]

for i, chain in enumerate(cleaned_chains, 1):
    route_seq = [str(driver_routes.get(bus, 'NA')) for bus in chain]
    chain_data.append({
        'Swap Chain': f'Chain {i}',
        'Routes': route_seq,
        'Vehicles': chain
    })
    
chains_df = pd.DataFrame(chain_data)


In [ ]:
chains_df

In [ ]:
assert assignments['From_Bus'].is_unique, "Duplicate drivers!"
assert assignments['To_Bus'].is_unique, "Duplicate pickups!"
